# InfCtx torch.compile performance uplift
The following trainer validation, is used to compare performance differences between the following optimizations

- torch native
- torch + JIT
- torch + torch.compile

It presumes that basic setup has been done as per
- `./baseline-setup.ipynb`

## Install the nightly build within conda
(Skip if you already have 2.0.2, or already done the setup)

For torch.compile, as of 8th July 2023, you will need the torch nightly build for several fixes we depend on. This is expected to be resolved when merged in for torch 2.0.2 (you will need to call this outside the notebook)

```bash
conda activate rwkv-infctx
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch-nightly -c nvidia
```

To simplify the benchmarking, we are intentionally only performing
- a 100 trainer/global_step
- of 10 gradient accumulation * gpu
- no checkpoint save to disk

This would (on a single GPU) perform the run over
- a 1000 data samples

## Configure and apply your preferred settings
( note you will need to rerun this cell, if you restart your env )

In [21]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
ENABLE_WANDB=False

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)

DEEPSPEED_STRAT: deepspeed_stage_2_offload
ENABLE_WANDB: False


## Run, and get baseline timing (no JIT / torch compile)

In [23]:
%%bash -s "$DEEPSPEED_STRAT" "$ENABLE_WANDB"

export RWKV_TORCH_COMPILE=0 
export RWKV_JIT_ON=0
if [ "$2" = "False" ]; then
    export WANDB_MODE="disabled"
fi

cd ../../RWKV-v4neo

python3 new_train.py fit \
    -c ../notebook/trainer-validation/config-baseline.yaml \
    --trainer.strategy="$1" \
    --trainer.logger.init_args.name="infctx-validation-baseline (OS environment vars: RWKV_JIT_ON=False, RWKV_TORCH_COMPILE=False)" \
    --trainer.max_epochs=-1 --trainer.max_steps:1000

WANDB: False


[2023-07-07 18:25:17,089] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model with : torch-native


Global seed set to 3941088705
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_128_bf16/build.ninja...
Building extension module wkv_128_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv_128_bf16...
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/connector.py:555: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard suppo

...

ninja: no work to do.


Loading extension module cpu_adam...


Time to load cpu_adam op: 2.3277952671051025 seconds
Rank: 0 partition count [1, 1, 1] and sizes[(1515008000, False), (49152, False), (49152, False)] 



  | Name   | Type       | Params
--------------------------------------
0 | emb    | Embedding  | 102 M 
1 | blocks | ModuleList | 1.3 B 
2 | ln_out | LayerNorm  | 4.1 K 
3 | head   | Linear     | 102 M 
--------------------------------------
1.5 B     Trainable params
0         Non-trainable params
1.5 B     Total params
6,060.425 Total estimated model params size (MB)
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/deepspeed/runtime/zero/stage_1_and_2.py:1828: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/conda/conda-bld/pytorch_1688627653114/work/torch/csrc/tensor/python_tensor.cpp:83.)
  overflow_gpu = get_accelerator().ByteTensor([overflow])
`Trainer.fit` stopped: `max_steps=2` reached.


Epoch 0:   0%|          | 4/5308 [01:01<22:28:21, 15.25s/it, v_num=10, train/loss=9.620]
